In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [2]:
def print_2d_matrix(matrix, decimals=3):
    for row in matrix:
        print(" ".join(f"{val:9.{decimals}f}" for val in row))

In [3]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [4]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# WFN data
wfndata = wfn2py(dirname+'WFN.h5')
wfnqdata = wfn2py(dirname+'WFNq.h5')

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

In [5]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 13294.15it/s]


In [6]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

Epsilon> q-pt index:   0%|          | 0/8 [00:00<?, ?it/s]

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 135.25it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [7]:
from qtm.gw.sigma import Sigma

sigma = Sigma.from_data(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    sigmainp=sigmainp,
    epsinp=epsinp,
    l_epsmats=epsilon.l_epsinv,
    rho=rho,
    vxc=vxc,
)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 18030.32it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x793c2d157790>
        * qpts = <qtm.gw.core.QPoints object at 0x793bf3966f10>
        * bare_coulomb_cutoff = 2.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 8
        * vcoul = <class 'list'> of length 8
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 8/8 [00:05<00:00,  1.52it/s]


In [8]:
from kernel import KernelMtxEl

q0 = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)

kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    q0=q0,
    l_epsmats=epsilon.l_epsinv,
    parallel=True,
)

In [9]:
data = kernelclass.kernel_mtxel()
exc = data["exc"]
head = data["head"]
body = data["body"]
wings = data["wings"]

In [10]:
numq = kernelclass.qpts.numq
numk = kernelclass.kpts.numk

In [11]:
ikf, ikc = np.random.randint(0, numk, size=2)
print(f"ikf, ikc = {ikf}, {ikc}")

ikf, ikc = 1, 0


In [12]:
from intkernel import InterpMtxEl

InterpClass = InterpMtxEl.from_BGW(
    wfn_finedata=wfndata,
    wfn_coarsedata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    kernel=kernelclass,
)

In [13]:
coeff_val = InterpClass.coeff_mtxel(ikf, ikf, "val")
coeff_con = InterpClass.coeff_mtxel(ikf, ikc, "con")


print(f"coeff_val is")
print_2d_matrix(coeff_val, decimals=3)

print(f"\n coeff_con is")
print_2d_matrix(coeff_con, decimals=3)

coeff_val is
1.000-0.000j 0.000+0.000j 0.000-0.000j -0.000+0.000j
0.000-0.000j 1.000-0.000j 0.000+0.000j 0.000+0.000j
0.000+0.000j 0.000-0.000j 1.000+0.000j -0.000-0.000j
-0.000-0.000j 0.000-0.000j -0.000+0.000j 1.000-0.000j

 coeff_con is
-0.316-0.240j -0.143-0.504j 0.161+0.172j -0.284+0.657j
0.702-0.375j -0.599-0.093j -0.008+0.003j 0.000+0.000j
0.177-0.113j 0.263+0.020j 0.866-0.370j -0.001-0.000j
-0.072+0.172j -0.212+0.125j 0.059-0.094j 0.934+0.142j


In [14]:
randomarr = np.random.rand(numk, 8)
print(f"\n randomarr is")
print_2d_matrix(randomarr, decimals=3)



 randomarr is
    0.794     0.530     0.726     0.202     0.071     0.925     0.387     0.213
    0.495     0.403     0.160     0.568     0.539     0.267     0.114     0.352
    0.149     0.721     0.560     0.930     0.650     0.623     0.978     0.394
    0.719     0.941     0.263     0.829     0.699     0.944     0.652     0.515
    0.241     0.713     0.644     0.186     0.689     0.628     0.347     0.212
    0.463     0.343     0.819     0.618     0.341     0.978     0.185     0.045
    0.025     0.483     0.298     0.398     0.938     0.830     0.613     0.720
    0.159     0.217     0.258     0.073     0.432     0.016     0.417     0.275


In [15]:
energyval = InterpClass.interp_energy(randomarr, "val")
energycon = InterpClass.interp_energy(randomarr, "con")


print(f"\n energyval is")
print_2d_matrix(energyval, decimals=3)

print(f"\n energycon is")
print_2d_matrix(energycon, decimals=3)


 energyval is
    0.202     0.726     0.530     0.794
    0.568     0.160     0.403     0.495
    0.930     0.560     0.721     0.149
    0.829     0.263     0.941     0.719
    0.186     0.644     0.713     0.241
    0.618     0.819     0.343     0.463
    0.398     0.298     0.483     0.025
    0.073     0.258     0.217     0.159

 energycon is
    0.071     0.925     0.387     0.213
    0.539     0.267     0.114     0.352
    0.650     0.623     0.978     0.394
    0.699     0.944     0.652     0.515
    0.689     0.628     0.347     0.212
    0.341     0.978     0.185     0.045
    0.938     0.830     0.613     0.720
    0.432     0.016     0.417     0.275


In [16]:
is_equal_val = np.allclose(np.flip(randomarr[:, :4], axis = -1), energyval)
is_equal_con = np.allclose(randomarr[:, 4:], energycon)

print(f"Is the interpolated valence energy equal to the random array? {is_equal_val}")
print(f"Is the interpolated conduction energy equal to the random array? {is_equal_con}")

Is the interpolated valence energy equal to the random array? True
Is the interpolated conduction energy equal to the random array? True


In [17]:
fine_kernel = InterpClass.interp_kernel(head, wings, body, exc, sigma.vcoul)
fine_kernel = fine_kernel[:, :, 0, 0, 0, 0]
print(f"\n fine_kernel is")

print_2d_matrix(fine_kernel, decimals=3)


 fine_kernel is
0.248-0.000j -0.151+0.575j -0.332+0.112j -0.187-0.336j -0.553-0.846j 0.044+0.151j 0.063-0.246j 0.013+0.002j
-0.151-0.575j 0.635+0.000j 0.166+0.252j -0.917+0.879j 0.198-0.219j -1.069+0.642j 0.183-0.001j -0.007-0.051j
-0.332-0.112j 0.166-0.252j 0.635-0.000j 0.228+1.239j -0.076-0.292j -0.008+0.183j 0.685-1.058j 0.002+0.001j
-0.187+0.336j -0.917-0.879j 0.228-1.239j 0.501+0.000j -0.183+0.013j 0.081+0.019j -0.061-0.057j -0.076+0.105j
-0.553+0.846j 0.198+0.219j -0.076+0.292j -0.183-0.013j 0.635-0.000j -1.194-0.363j 0.860+0.937j -0.032+0.038j
0.044-0.151j -1.069-0.642j -0.008-0.183j 0.081-0.019j -1.194+0.363j 0.501+0.000j -0.072-0.042j -0.001+0.004j
0.063+0.246j 0.183+0.001j 0.685+1.058j -0.061+0.057j 0.860-0.937j -0.072+0.042j 0.501-0.000j 0.017+0.132j
0.013-0.002j -0.007+0.051j 0.002-0.001j -0.076-0.105j -0.032-0.038j -0.001-0.004j 0.017-0.132j 0.634-0.000j
